# CRISP-DM Giai đoạn 6: Triển khai & Dự đoán (Deployment)

**Mục tiêu:**
1.  Xây dựng lớp `ChurnPredictor` để đóng gói logic dự đoán.
2.  Mô phỏng **Single Prediction**: Dự đoán cho một khách hàng đơn lẻ (dùng cho form trên web).
3.  Mô phỏng **Batch Prediction**: Dự đoán cho một danh sách khách hàng (dùng cho báo cáo định kỳ).

[Image of data pipeline architecture]

In [ ]:
import joblib
import os
import pandas as pd
import numpy as np

# Cấu hình đường dẫn (Điều chỉnh relative path cho phù hợp với Notebook)
CURRENT_DIR = os.getcwd()
MODELS_DIR = os.path.join(CURRENT_DIR, 'models')
MODEL_PATH = os.path.join(MODELS_DIR, 'best_model.pkl') # Đảm bảo tên file khớp với bước trước

print(f"Đường dẫn model dự kiến: {MODEL_PATH}")

# --- Helper: Tạo dummy model nếu chưa có (Để Notebook chạy được ngay) ---
# (Chỉ dùng cho mục đích demo nếu bạn chưa chạy bước Training trước đó)
if not os.path.exists(MODEL_PATH):
    print("⚠️ CẢNH BÁO: Không tìm thấy file model thực tế. Đang tạo Dummy Model để demo...")
    from sklearn.dummy import DummyClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    
    # Tạo dummy pipeline
    os.makedirs(MODELS_DIR, exist_ok=True)
    dummy_model = Pipeline([
        ('preprocessor', ColumnTransformer([
            ('num', StandardScaler(), ['tenure', 'MonthlyCharges']),
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['Contract', 'InternetService', 'PaymentMethod'])
        ])),
        ('classifier', DummyClassifier(strategy="uniform"))
    ])
    # Fake fit
    X_dummy = pd.DataFrame({
        'tenure': [1, 10], 'MonthlyCharges': [50, 100], 
        'Contract': ['Month-to-month', 'Two year'], 
        'InternetService': ['DSL', 'Fiber optic'], 
        'PaymentMethod': ['Electronic check', 'Mailed check']
    })
    y_dummy = [0, 1]
    dummy_model.fit(X_dummy, y_dummy)
    joblib.dump(dummy_model, MODEL_PATH)
    print("✅ Đã tạo Dummy Model thành công.")

In [ ]:
class ChurnPredictor:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load model pipeline từ disk"""
        if os.path.exists(self.model_path):
            self.model = joblib.load(self.model_path)
            print(f"✅ Đã tải model từ: {self.model_path}")
        else:
            raise FileNotFoundError(f"❌ Không tìm thấy model tại {self.model_path}")

    def predict_one(self, data_dict):
        """Dự đoán cho 1 mẫu dữ liệu (Input: Dictionary)"""
        if not self.model:
            raise RuntimeError("Model chưa được khởi tạo.")

        # Chuyển Dict thành DataFrame 1 dòng
        df = pd.DataFrame([data_dict])
        
        # Loại bỏ cột ID nếu có
        if 'customerID' in df.columns:
            df = df.drop(columns=['customerID'])
        
        # Dự đoán
        prediction = self.model.predict(df)[0]
        probability = self.model.predict_proba(df)[0][1] # Lấy xác suất của lớp 1 (Churn)
        
        return {
            "prediction": int(prediction),
            "churn_probability": round(float(probability), 4),
            "label": "RỜI BỎ (Churn)" if prediction == 1 else "Ở LẠI (No Churn)",
            "risk_level": "CAO" if probability > 0.7 else ("TRUNG BÌNH" if probability > 0.4 else "THẤP")
        }

    def predict_batch(self, df):
        """Dự đoán cho nhiều mẫu (Input: DataFrame)"""
        if not self.model:
            raise RuntimeError("Model chưa được khởi tạo.")
        
        work_df = df.copy()
        if 'customerID' in work_df.columns:
            work_df = work_df.drop(columns=['customerID'])
            
        predictions = self.model.predict(work_df)
        probabilities = self.model.predict_proba(work_df)[:, 1]
        
        # Gắn kết quả vào DataFrame gốc để báo cáo
        results_df = df.copy()
        results_df['Prediction'] = predictions
        results_df['Probability'] = probabilities.round(4)
        results_df['Risk_Level'] = pd.cut(results_df['Probability'], 
                                          bins=[0, 0.4, 0.7, 1.0], 
                                          labels=['Thấp', 'Trung bình', 'Cao'])
        return results_df

# Khởi tạo đối tượng dự đoán
try:
    predictor = ChurnPredictor(MODEL_PATH)
except Exception as e:
    print(f"Lỗi khởi tạo: {e}")

## 1. Dự đoán Đơn lẻ (Single Prediction)
Giả lập một khách hàng mới với thông tin hợp đồng cụ thể để kiểm tra khả năng rời bỏ.

In [ ]:
# Giả lập dữ liệu nhập từ Form
new_customer = {
    "customerID": "TEST_001",
    "tenure": 5,                        # Mới dùng 5 tháng
    "InternetService": "Fiber optic",   # Dùng cáp quang
    "Contract": "Month-to-month",       # Hợp đồng từng tháng (Rủi ro cao)
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 95.50             # Cước phí cao
}

print("--- THÔNG TIN KHÁCH HÀNG ---")
print(new_customer)
print("\n--- KẾT QUẢ DỰ ĐOÁN ---")

try:
    result = predictor.predict_one(new_customer)
    
    # Hiển thị kết quả đẹp mắt
    for key, value in result.items():
        print(f"{key:<20}: {value}")
        
    # Logic cảnh báo
    if result['prediction'] == 1:
        print("\n⚠️ KHUYẾN NGHỊ: Khách hàng có nguy cơ rời bỏ cao. Cần gửi mã giảm giá ngay!")
    else:
        print("\n✅ KHUYẾN NGHỊ: Khách hàng ổn định.")
        
except Exception as e:
    print(f"Lỗi dự đoán: {e}")

## 2. Dự đoán Hàng loạt (Batch Prediction)
Tạo danh sách 5 khách hàng giả lập và phân loại mức độ rủi ro cho từng người.

In [ ]:
# Tạo danh sách khách hàng giả lập
batch_data = pd.DataFrame([
    {"customerID": "C001", "tenure": 72, "InternetService": "No", "Contract": "Two year", "PaymentMethod": "Mailed check", "MonthlyCharges": 20.0},
    {"customerID": "C002", "tenure": 2, "InternetService": "Fiber optic", "Contract": "Month-to-month", "PaymentMethod": "Electronic check", "MonthlyCharges": 105.0},
    {"customerID": "C003", "tenure": 25, "InternetService": "DSL", "Contract": "One year", "PaymentMethod": "Bank transfer", "MonthlyCharges": 65.0},
    {"customerID": "C004", "tenure": 1, "InternetService": "Fiber optic", "Contract": "Month-to-month", "PaymentMethod": "Electronic check", "MonthlyCharges": 70.0},
    {"customerID": "C005", "tenure": 45, "InternetService": "DSL", "Contract": "One year", "PaymentMethod": "Credit card", "MonthlyCharges": 55.0},
])

print("Đang xử lý lô dữ liệu...")
results_df = predictor.predict_batch(batch_data)

# Hiển thị kết quả sử dụng màu sắc (Highlight)
def highlight_risk(val):
    color = 'red' if val == 'Cao' else ('orange' if val == 'Trung bình' else 'green')
    return f'color: {color}; font-weight: bold'

# Display DataFrame với style
display(results_df[['customerID', 'tenure', 'Contract', 'Prediction', 'Probability', 'Risk_Level']]
        .style.applymap(highlight_risk, subset=['Risk_Level']))